# Index

* [Data Preparation](#data_prepa)
    - [Data Cleaning](#data_cleaning)
        - [Missing Values](#missing_values)
        - [Outliers](#outliers)
    - [Feature Engineering](#feature_engineering)

## Data Preparation

We'll create a pipeline with scikit learn to perform the data preparation including the transformations identified in EDA, recall here:
* Input missing values
* Create new features:
    - New features with the mean values of meteorological variables for every Numerical Weather Predictor (`U`, `V`, `T`, `CLCT`)
    - Wind velocity module (`w_vel`)
    - Wind direction (`w_dir`)
    - Wind velocity escaled to heigth of the turbine (`w_vel_esc`). We can calculate it by using 
    
        $$u(z) = u(z_0)\left(\frac{z}{z_0}\right)^{\alpha},$$ 
      
      with $\alpha = 1/7$, $z_0 = 100$ or $10$ meters, depending on the Wild Farm, and $z = 50$ m, the height of the turbines.
    - Date time future enconding to capture seasonality (`month`, `day_of_month`, `hour`)
    - Cyclical enconding for wind direction and date time features.
* Stardard Scaling of variables
* Outlier treatment (to define, using the extra data in order to identify anomalies).
* Feature selection

### List of hypotesis 
1. Meteorological forecasts accuracy and reliability are the highest for Forecast Day `D` and Run `18h` (the shorter the gap time between NWP Run and the time forecasted, the higher the realiability).

In [1]:
# Libraries

%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import pandas as pd
import numpy as np
import datetime as dt
import gc
import missingno as msno
import pandas_profiling
import statsmodels as sm
from statsmodels.tsa.seasonal import seasonal_decompose
import random

from src.functions import data_import as dimp
from src.functions import data_exploration as dexp
from src.functions import data_transformation as dtr

#visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as pty

import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
cf.set_config_file(offline=True)

In [290]:
# Load data (they were stored in notebook 00-vcp-frame-problem-and-get-data)
%store -r X_train_2
%store -r Y_train_2
%store -r X_test_2
%store -r Y_test_2

In [291]:
# Select rows only for WF1
X_train = X_train_2[X_train_2.WF == 'WF1']
X_test = X_test_2[X_test_2.WF == 'WF1']
y_train = Y_train_2[Y_train_2.ID.isin(X_train.ID)]
y_test = Y_test_2[Y_test_2.ID.isin(X_test.ID)]

In [292]:
# Keep IDs separately
ID_X_train = X_train['ID']
ID_X_test = X_test['ID']
ID_y_train = y_train['ID']
ID_y_test = y_test['ID']

### Data Cleaning
<a id="data_cleaning"></a>

#### Missing Values
<a id="missing_values"></a>

We'll use whenever is it possible, data from forecast day `D` and run `18h` for every `NWP` as they're the most recent data available. When there're missing values, we'll use the corresponding data from previous days to fill in the gap, sequencely.

Moreover, `NWP2` and `NWP3` provides data every 3 hours, instead of every 1 hour, as `NWP1` and `NWP4` do. We have then the following missing values to input:
* `NWP1`: day 13/07/2018 is fully missing. We'll use `D-1` forecasts to input these values.
* `NWP4`: day 15/05/2018 is fully missing. We'll use `D-1` forecasts to input these values.
* `NWP2` and `NWP3`: we will interpolate these values, missing three hours in between every data.

In [293]:
X_train.loc[X_train['NWP3_00h_D_U'].isna(), 'Time']

0      2018-05-01 01:00:00
1      2018-05-01 02:00:00
3      2018-05-01 04:00:00
4      2018-05-01 05:00:00
6      2018-05-01 07:00:00
               ...        
6183   2019-01-13 17:00:00
6185   2019-01-13 19:00:00
6186   2019-01-13 20:00:00
6188   2019-01-13 22:00:00
6189   2019-01-13 23:00:00
Name: Time, Length: 4127, dtype: datetime64[ns]

In [294]:
# We'll add new columns NWPX_<met_var> without missing values
# new cols: NWP1_U, NWP1_V, NWP1_T, NWP2_U, NWP2_V, NWP3_U, NWP3_V, NWP3_T, NWP4_U, NWP4_V, NWP4_CLCT

new_cols = ['NWP1_U','NWP1_V','NWP1_T','NWP2_U',
            'NWP2_V','NWP3_U','NWP3_V','NWP3_T',
            'NWP4_U','NWP4_V','NWP4_CLCT']

def add_new_cols(new_cols, df):
    for col in new_cols:
        df[col] = np.nan        

In [295]:
add_new_cols(new_cols, X_train)

In [296]:
def input_missing_values(df, cols):
    regex = 'NWP(?P<NWP>\d{1})_(?P<run>\d{2}h)_(?P<fc_day>D\W?\d?)_(?P<weather_var>\w{1,4})'
    p = re.compile(regex)  
    
    NWP_met_vars_dict = {
        '1': ['U','V','T'],
        '2': ['U','V'],
        '3': ['U','V','T'],
        '4': ['U','V','CLCT']
    }
    
    for col in reversed(cols):
        m = p.match(col)
        col_name = 'NWP' + m.group('NWP') + '_' +  m.group('run') + '_' + m.group('fc_day') + '_' + m.group('weather_var')
        nwp = m.group('NWP')

        for key, value in NWP_met_vars_dict.items():
            for i in value:
                if m.group('NWP') == key and m.group('weather_var') == i:
                    df['NWP'+ key + '_' + i] = df['NWP'+ key + '_' + i].fillna(df[col_name])
    
    return df

In [297]:
cols = X_train.columns[3:-11] 
X_train = input_missing_values(X_train, cols)

In [298]:
X_train[new_cols].head()

,NWP1_U,NWP1_V,NWP1_T,NWP2_U,NWP2_V,NWP3_U,NWP3_V,NWP3_T,NWP4_U,NWP4_V,NWP4_CLCT
0,-2.248047,-3.257812,286.50,NaN,NaN,NaN,NaN,NaN,1.254883,-0.289795,82.5625
1,-2.433594,-1.446289,286.25,NaN,NaN,NaN,NaN,NaN,2.490234,-0.413330,100.0000
2,3.365234,-3.060547,285.75,2.611328,-2.341797,-1.149414,-2.275391,286.0,0.997070,-1.415039,98.3750
3,3.707031,-6.218750,284.75,NaN,NaN,NaN,NaN,NaN,0.689453,-0.961426,94.8750
4,3.812500,-5.445312,284.50,NaN,NaN,NaN,NaN,NaN,0.291016,-0.294922,95.8750


Now we'll interpolate in NWP2 and NWP3 in order to obtain the remaining missing values.

In [300]:
X_train_cpy = X_train.copy()
col_list = ['NWP2_U','NWP2_V','NWP3_U','NWP3_V','NWP3_T']
X_train_cpy.index = X_train_cpy['Time']
del X_train_cpy['Time']
    
for var in col_list:
    X_train_cpy[var].interpolate(
        method='time', 
        inplace=True,
        limit=2,
        limit_direction='both'
    )

    
X_train_cpy.reset_index(inplace=True)


In [301]:
X_train[new_cols].head(20)

,NWP1_U,NWP1_V,NWP1_T,NWP2_U,NWP2_V,NWP3_U,NWP3_V,NWP3_T,NWP4_U,NWP4_V,NWP4_CLCT
0,-2.248047,-3.257812,286.50,NaN,NaN,NaN,NaN,NaN,1.254883,-0.289795,82.56250
1,-2.433594,-1.446289,286.25,NaN,NaN,NaN,NaN,NaN,2.490234,-0.413330,100.00000
2,3.365234,-3.060547,285.75,2.611328,-2.341797,-1.149414,-2.275391,286.0,0.997070,-1.415039,98.37500
3,3.707031,-6.218750,284.75,NaN,NaN,NaN,NaN,NaN,0.689453,-0.961426,94.87500
4,3.812500,-5.445312,284.50,NaN,NaN,NaN,NaN,NaN,0.291016,-0.294922,95.87500
5,4.640625,-4.996094,284.00,4.312500,-3.666016,0.741211,-3.613281,285.0,0.265137,-0.170654,65.37500
6,2.677734,-5.796875,284.25,NaN,NaN,NaN,NaN,NaN,1.230469,-0.128540,75.18750
7,0.777832,-5.265625,285.00,NaN,NaN,NaN,NaN,NaN,2.703125,-0.072815,90.93750
8,1.395508,-4.359375,286.00,0.752441,-3.923828,1.257812,-3.650391,286.0,3.205078,-0.791504,100.00000
9,1.905273,-2.871094,287.25,NaN,NaN,NaN,NaN,NaN,3.257812,-2.164062,96.87500


In [302]:
X_train_cpy[new_cols].head(20)

,NWP1_U,NWP1_V,NWP1_T,NWP2_U,NWP2_V,NWP3_U,NWP3_V,NWP3_T,NWP4_U,NWP4_V,NWP4_CLCT
0,-2.248047,-3.257812,286.50,2.611328,-2.341797,-1.149414,-2.275391,286.000000,1.254883,-0.289795,82.56250
1,-2.433594,-1.446289,286.25,2.611328,-2.341797,-1.149414,-2.275391,286.000000,2.490234,-0.413330,100.00000
2,3.365234,-3.060547,285.75,2.611328,-2.341797,-1.149414,-2.275391,286.000000,0.997070,-1.415039,98.37500
3,3.707031,-6.218750,284.75,3.178385,-2.783203,-0.519206,-2.721354,285.666667,0.689453,-0.961426,94.87500
4,3.812500,-5.445312,284.50,3.745443,-3.224609,0.111003,-3.167318,285.333333,0.291016,-0.294922,95.87500
5,4.640625,-4.996094,284.00,4.312500,-3.666016,0.741211,-3.613281,285.000000,0.265137,-0.170654,65.37500
6,2.677734,-5.796875,284.25,3.125814,-3.751953,0.913411,-3.625651,285.333333,1.230469,-0.128540,75.18750
7,0.777832,-5.265625,285.00,1.939128,-3.837891,1.085612,-3.638021,285.666667,2.703125,-0.072815,90.93750
8,1.395508,-4.359375,286.00,0.752441,-3.923828,1.257812,-3.650391,286.000000,3.205078,-0.791504,100.00000
9,1.905273,-2.871094,287.25,1.609049,-3.763021,1.516927,-3.821615,286.333333,3.257812,-2.164062,96.87500


In [303]:
X_train = X_train_cpy

Let's add columns `U`, `V`, `T`, `CLCT` by calculating the mean for each Numerical Weather Predictor

In [304]:
X_train['U'] = (X_train.NWP1_U + X_train.NWP2_U + X_train.NWP3_U + X_train.NWP4_U)/4
X_train['V'] = (X_train.NWP1_V + X_train.NWP2_V + X_train.NWP3_V + X_train.NWP4_V)/4
X_train['T'] = (X_train.NWP1_T + X_train.NWP3_T)/2
X_train['CLCT'] = (X_train.NWP4_CLCT)

In [305]:
X_train.head()

,Time,ID,WF,NWP1_00h_D-2_U,NWP1_00h_D-2_V,NWP1_00h_D-2_T,NWP1_06h_D-2_U,NWP1_06h_D-2_V,NWP1_06h_D-2_T,NWP1_12h_D-2_U,...,NWP3_U,NWP3_V,NWP3_T,NWP4_U,NWP4_V,NWP4_CLCT,U,V,T,CLCT
0,2018-05-01 01:00:00,1,WF1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.149414,-2.275391,286.000000,1.254883,-0.289795,82.5625,0.117188,-2.041199,286.250000,82.5625
1,2018-05-01 02:00:00,2,WF1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.149414,-2.275391,286.000000,2.490234,-0.413330,100.0000,0.379639,-1.619202,286.125000,100.0000
2,2018-05-01 03:00:00,3,WF1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.149414,-2.275391,286.000000,0.997070,-1.415039,98.3750,1.456055,-2.273193,285.875000,98.3750
3,2018-05-01 04:00:00,4,WF1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.519206,-2.721354,285.666667,0.689453,-0.961426,94.8750,1.763916,-3.171183,285.208333,94.8750
4,2018-05-01 05:00:00,5,WF1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.111003,-3.167318,285.333333,0.291016,-0.294922,95.8750,1.989990,-3.033040,284.916667,95.8750


#### Outliers
<a id="outliers"></a>

In [101]:
f = lambda x : round(x,1)
X_train.NWP1_T.apply(f)

0       286.5
1       286.2
2       285.8
3       284.8
4       284.5
        ...  
6185    283.2
6186    283.2
6187    283.2
6188    283.0
6189    282.8
Name: NWP1_T, Length: 6190, dtype: float64

### Feature Engineering

In [111]:
X_train_2.NWP1_00h_D_T

0        286.50
1        286.25
2        285.75
3        284.75
4        284.50
          ...  
37321    277.25
37322    276.50
37323    276.00
37324    276.25
37325    276.50
Name: NWP1_00h_D_T, Length: 37081, dtype: float16

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from metpy import calc
from metpy.units import units

# function to obtain the module of wind velocity
get_wind_velmod = lambda x : float(calc.wind_speed(
    x.U * units.meter/units.second, 
    x.V * units.meter/units.second,
).magnitude)

# function to obtain the wind direction
get_wind_dir = lambda x : float(calc.wind_direction(
    x.U * units.meter/units.second, 
    x.V * units.meter/units.second, 
    convention="from"
).magnitude)


# class to ad the new features
class NewFeatureAdder(BaseEstimator, TransformerMixin):

    def __init__(self):
        return self

    def fit(self, documents, y=None):
        return self

    def transform(self, x_dataset):
        # Velocity features
        x_dataset['w_vel'] = x_dataset.apply(get_wind_velmod, axis=1)
        x_dataset['w_dir'] = x_dataset.apply(get_wind_dir, axis=1)
        
        
        if self.add_time_feat:
            # Creamos atributos derivados de la fecha
            x_dataset['month'] = x_dataset['time'].dt.month
            x_dataset['hour'] = x_dataset['time'].dt.hour
            x_dataset['day_of_week'] = x_dataset['time'].dt.dayofweek
            x_dataset['day_of_month'] = x_dataset['time'].dt.day
            
        if self.add_cyclic_feat:
            # Hour
            x_dataset['hr_sin'] = np.sin(x_dataset['hour'] * (2.* np.pi / 24))
            x_dataset['hr_cos'] = np.cos(x_dataset['hour'] * (2.* np.pi / 24))

            # Day of the week
            x_dataset['wday_sin'] = np.sin(x_dataset['day_of_week'] * (2.* np.pi / 7))
            x_dataset['wday_cos'] = np.cos(x_dataset['day_of_week'] * (2.* np.pi / 7))

            # Month
            x_dataset['mnth_sin'] = np.sin((x_dataset['month']-1) * (2.* np.pi / 12))
            x_dataset['mnth_cos'] = np.cos((x_dataset['month']-1) * (2.* np.pi / 12))
            
            # Wind direction
            x_dataset['wdir_sin'] = np.sin(x_dataset['w_dir'] * (2.* np.pi / 360))
            x_dataset['wdir_cos'] = np.cos(x_dataset['w_dir'] * (2.* np.pi / 360))
            
             
        return x_dataset


In [76]:
X_train.NWP1_T.isna().describe()

count      6190
unique        1
top       False
freq       6190
Name: NWP1_T, dtype: object